<a href="https://colab.research.google.com/github/Chandrika2004/FMML_LAB-MODULES/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1st question ANS

Averaging validation accuracy across multiple splits is a common technique used in machine learning to obtain more robust and consistent performance estimates for a model. This technique is typically used in the context of k-fold cross-validation, where the dataset is divided into k subsets (folds). The model is trained and validated k times, each time using a different fold as the validation set and the remaining folds as the training set. The validation accuracies obtained from each of these k runs are then averaged to provide a more reliable estimate of model performance.

Here are some reasons why averaging validation accuracy across multiple splits can give more consistent results:

Reduced Variance: By using multiple splits, you reduce the impact of random variation in the data. Different splits can have different characteristics, and averaging helps to smooth out these variations.

Better Generalization: Averaging over multiple splits helps ensure that your model's performance estimate is not overly dependent on a particular random split of the data. It provides a more representative assessment of how well the model is likely to perform on unseen data.

Robustness to Data Imbalances: If your dataset has class imbalances or other irregularities, using multiple splits can help ensure that each fold has a roughly similar distribution of classes, which can be important for accurate evaluation.

Model Stability: It can help you identify if your model is stable across different subsets of the data. If your model performs consistently well across all splits, it suggests that it is robust and less likely to overfit to specific data instances.

Reduced Bias: Averaging over multiple splits reduces the potential for bias that might arise from a single, particularly challenging or easy validation split.

However, it's important to note that the number of splits (k) can affect the trade-off between computational cost and result stability. Larger values of k require more computation but can provide a more accurate estimate of performance. Smaller values of k may be computationally efficient but can yield more variable estimates.

In summary, averaging validation accuracy across multiple splits is a valuable technique to obtain more consistent and reliable estimates of model performance, especially when dealing with limited data or when you want to assess your model's robustness.





Regenerate


2 nd question answer

Averaging validation accuracy across multiple splits, such as in k-fold cross-validation, provides a more accurate estimate of how well your model is likely to perform on unseen data compared to using a single train-test split. However, it's important to understand that the validation accuracy obtained through cross-validation is still an estimate of test performance, not the actual test accuracy.

Here's why it provides a more accurate estimate:

Reduced Bias: Using multiple splits helps reduce the potential bias that can arise from a single, possibly unrepresentative train-test split. With k-fold cross-validation, each data point has the opportunity to be in the validation set exactly once, which helps to ensure a more balanced representation of the data.

Better Generalization: Cross-validation assesses the model's ability to generalize by training and validating on different subsets of the data. This provides a more robust estimate of how well the model is expected to perform on unseen data.

Reduced Variance: Averaging the results from multiple splits helps reduce the impact of random variation in the data, leading to a more stable estimate of model performance.

While cross-validation provides a more accurate estimate of test performance compared to a single train-test split, it's essential to keep in mind that it's still an estimate. The actual performance on completely unseen data (your true test set) may still differ somewhat from the cross-validation results because real-world data can have unique characteristics that were not captured in the cross-validation splits.

To obtain the most accurate estimate of test accuracy, you should reserve a separate, completely independent dataset that has not been used in any way during model development or cross-validation. This held-out test set can provide a more accurate reflection of how your model will perform on new, unseen data. Cross-validation is a valuable technique for estimating performance, model selection, and tuning hyperparameters, but it doesn't replace the need for a final evaluation on a truly independent test set.





Regenerate


3 rd question ans

In the context of machine learning, the term "number of iterations" can refer to different things depending on the specific algorithm or technique you're using. Let's discuss two common scenarios where the number of iterations can impact the estimate:

Training Iterations (Epochs):

For iterative training algorithms like gradient descent used in neural networks, the number of training iterations (often referred to as epochs) can affect the model's performance estimate.
In general, increasing the number of training iterations can help the model learn better and potentially improve its performance. However, this improvement is subject to diminishing returns. After a certain point, increasing the number of iterations may lead to overfitting on the training data, where the model becomes too specialized and performs poorly on unseen data (validation or test data).
To avoid overfitting, it's common to monitor the model's performance on a validation set during training and stop when the performance on the validation set starts to degrade.
Cross-Validation Folds:

In k-fold cross-validation, the number of iterations refers to the value of k, which determines how many times the dataset is split into training and validation subsets.
Increasing the number of cross-validation folds (i.e., using a larger k) can potentially lead to a better estimate of model performance. With a larger k, you get more training and validation sets, which can result in a more stable and accurate estimate of model performance.
However, using a larger k also increases the computational cost because you need to train and validate the model k times. It's essential to strike a balance between computational resources and the quality of the estimate.
In summary, the impact of the number of iterations on the estimate depends on the specific context:

For training iterations in machine learning models, increasing the number of iterations can improve the model's performance up to a point, but there's a risk of overfitting if you overdo it.
For cross-validation folds, increasing the number of folds can lead to a more accurate estimate of model performance, but it comes at the cost of increased computational resources. You should choose an appropriate value for k based on your available resources and the need for a reliable performance estimate.




Regenerate


4 th question ans

Increasing the number of iterations can help improve model performance to some extent, even with a small training dataset, but there are limitations to how much improvement you can achieve solely by increasing iterations. The impact of increasing iterations on model performance depends on several factors, including the complexity of the model, the nature of the data, and the potential for overfitting. Let's explore this in more detail:

Model Complexity: If you have a simple model with a small number of parameters, increasing the number of training iterations may help the model converge to a better solution. However, complex models may require a larger and more diverse training dataset to generalize effectively.

Overfitting: Increasing the number of training iterations without an adequate amount of data can lead to overfitting. Overfitting occurs when a model becomes too specialized in the training data and performs poorly on unseen data. More iterations can allow a complex model to fit the noise in the data, resulting in poor generalization.

Data Quality: The quality and representativeness of the training data also play a crucial role. If your small training dataset is noisy or not representative of the target population, increasing iterations may not significantly improve performance.

Validation Dataset: Increasing iterations is not a direct solution to address issues with a small validation dataset. A small validation dataset can lead to unstable estimates of model performance, making it challenging to assess how well the model will generalize. It's generally advisable to have a reasonably sized validation dataset for robust performance evaluation.

In situations where you have a very small training dataset, and increasing the dataset size is not feasible, consider the following strategies:

Regularization: Apply regularization techniques such as L1 or L2 regularization to mitigate overfitting and improve generalization.

Simpler Models: Use simpler models with fewer parameters that are less prone to overfitting when data is limited.

Data Augmentation: If possible, apply data augmentation techniques to artificially increase the effective size of your training dataset by creating variations of existing data points.

Transfer Learning: If you have access to pre-trained models on a related task or domain, consider using transfer learning to leverage knowledge from those models.

Ensemble Methods: Combine the predictions of multiple models, such as bagging or boosting, to improve performance and reduce overfitting.

Stratified Sampling: When performing cross-validation, ensure that each fold maintains the same class distribution as the overall dataset to prevent biased estimates.

While increasing iterations can be a part of the solution, it's essential to consider it in conjunction with other techniques to address the challenges of working with a small training dataset and ensure that your model generalizes well to unseen data.




